In [1]:
# # Enable autoreload
# %load_ext autoreload
# %autoreload 2

In [2]:
import numpy as np
import os
import torch
import torch.nn as nn
import time
import pandas as pd
from scipy.stats import pearsonr

In [3]:
from model.util import Normalizer
from model.database_util import get_hist_file, get_job_table_sample, collator
from model.model import QueryFormer
from model.database_util import Encoding
from model.dataset import PlanTreeDataset
from model.trainer import eval_workload, train

In [4]:
data_path = './data/imdb/'

In [18]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 128
    lr = 0.001
    # epochs = 200
    epochs = 100
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    device = 'cuda:0'
    newpath = './results/full/cost/'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [6]:
# >>> df=pd.read_csv('histogram_string.csv')
# >>> df
#              table           column                                               freq                                               bins         table_column
# 0            title  production_year  0000000000000000000000000000000000000000000000...  [1880 1913 1923 1942 1955 1960 1964 1968 1971 ...    t.production_year
# 1            title          kind_id  0000000000000000a7d0a5b13fc7d03f6249099dec65a2...  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 3 3 4 4 4 7 7 7...            t.kind_id
# 2  movie_companies       company_id  000000000000000088995af55640403f993ed7b99d4f42...  [     1      6     19     27     68    133    ...        mc.company_id
# 3  movie_companies  company_type_id   0000000000000000d97131b09c41df3f1347e7a7315fe03f  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...   mc.company_type_id
# 4        cast_info          role_id  000000000000000060dbffdfb15fd63f07c2be4c3951ca...  [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ...           ci.role_id
# 5    movie_keyword       keyword_id  000000000000000079ee79d33c026a3f6235e2d63fb0c0...  [     1     77    132    230    331    347    ...        mk.keyword_id
# 6        cast_info        person_id  00000000000000000000000000000000972af4b705a06d...  [      2   77446  145798  212750  281691  3472...         ci.person_id
# 7   movie_info_idx     info_type_id  0000000000000000000000000000000000000000000000...  [ 99  99  99  99  99  99  99  99  99  99  99  ...  mi_idx.info_type_id
# 8       movie_info     info_type_id  000000000000000056a15855d2aeab3f075df65ac53db6...  [  1   1   1   2   2   2   2   2   3   3   3  ...      mi.info_type_id

In [7]:
hist_file = get_hist_file(data_path + 'histogram_string.csv')
cost_norm = Normalizer(-3.61192, 12.290855)
card_norm = Normalizer(1,100)

/home/wuy/DB/performance_estimation/QueryFormer/model/database_util.py:92: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  hist_file['bins'][rid] = \
/home/wuy/DB/performance_estimation/QueryFormer/model/database_util.py:92: FutureWarning: Cha

In [8]:
encoding_ckpt = torch.load('checkpoints/encoding.pt')
encoding = encoding_ckpt['encoding']
checkpoint = torch.load('checkpoints/cost_model.pt', map_location='cpu')

In [9]:
from model.util import seed_everything
seed_everything()

In [10]:
model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = True, use_hist = True, \
                 pred_hid = args.pred_hid
                )

In [19]:
_ = model.to(args.device)

In [12]:
to_predict = 'cost'

In [ ]:
# train_plan_part0.csv:
#         id                                               json
# 0        0  {"Plan": {"Node Type": "Gather", "Parallel Awa...
# 1        1  {"Plan": {"Node Type": "Seq Scan", "Parallel A...
# 2        2  {"Plan": {"Node Type": "Seq Scan", "Parallel A...
# 3        3  {"Plan": {"Node Type": "Gather", "Parallel Awa...
# 4        4  {"Plan": {"Node Type": "Bitmap Heap Scan", "Pa...
# ...    ...                                                ...
# 4995  4995  {"Plan": {"Node Type": "Index Scan", "Parallel...
# 4996  4996  {"Plan": {"Node Type": "Gather", "Parallel Awa...
# 4997  4997  {"Plan": {"Node Type": "Gather", "Parallel Awa...
# 4998  4998  {"Plan": {"Node Type": "Gather", "Parallel Awa...
# 4999  4999  {"Plan": {"Node Type": "Nested Loop", "Paralle...

# >>> df['json'][0]
# '{"Plan": {"Node Type": "Gather", "Parallel Aware": false, "Startup Cost": 23540.58, "Total Cost": 154548.95, "Plan Rows": 567655, "Plan Width": 119, "Actual Startup Time": 386.847, "Actual Total Time": 646.972, "Actual Rows": 283812, "Actual Loops": 1, "Workers Planned": 2, "Workers Launched": 2, "Single Copy": false, "Plans": [{"Node Type": "Hash Join", "Parent Relationship": "Outer", "Parallel Aware": true, "Join Type": "Inner", "Startup Cost": 22540.58, "Total Cost": 96783.45, "Plan Rows": 236523, "Plan Width": 119, "Actual Startup Time": 369.985, "Actual Total Time": 518.487, "Actual Rows": 94604, "Actual Loops": 3, "Inner Unique": false, "Hash Cond": "(t.id = mi_idx.movie_id)", "Workers": [], "Plans": [{"Node Type": "Seq Scan", "Parent Relationship": "Outer", "Parallel Aware": true, "Relation Name": "title", "Alias": "t", "Startup Cost": 0.0, "Total Cost": 49166.46, "Plan Rows": 649574, "Plan Width": 94, "Actual Startup Time": 0.366, "Actual Total Time": 147.047, "Actual Rows": 514421, "Actual Loops": 3, "Filter": "(kind_id = 7)", "Rows Removed by Filter": 328349, "Workers": []}, {"Node Type": "Hash", "Parent Relationship": "Inner", "Parallel Aware": true, "Startup Cost": 15122.68, "Total Cost": 15122.68, "Plan Rows": 383592, "Plan Width": 25, "Actual Startup Time": 103.547, "Actual Total Time": 103.547, "Actual Rows": 306703, "Actual Loops": 3, "Hash Buckets": 65536, "Original Hash Buckets": 65536, "Hash Batches": 32, "Original Hash Batches": 32, "Peak Memory Usage": 1920, "Workers": [], "Plans": [{"Node Type": "Seq Scan", "Parent Relationship": "Outer", "Parallel Aware": true, "Relation Name": "movie_info_idx", "Alias": "mi_idx", "Startup Cost": 0.0, "Total Cost": 15122.68, "Plan Rows": 383592, "Plan Width": 25, "Actual Startup Time": 0.28, "Actual Total Time": 54.382, "Actual Rows": 306703, "Actual Loops": 3, "Filter": "(info_type_id > 99)", "Rows Removed by Filter": 153308, "Workers": []}]}]}]}, "Planning Time": 2.382, "Triggers": [], "Execution Time": 654.241}'

In [13]:
imdb_path = './data/imdb/'
dfs = []  # list to hold DataFrames
# SQ: added
for i in range(2):
#for i in range(18):
    file = imdb_path + 'plan_and_cost/train_plan_part{}.csv'.format(i)
    df = pd.read_csv(file)
    dfs.append(df)

full_train_df = pd.concat(dfs)

val_dfs = []  # list to hold DataFrames
for i in range(18,20):
    file = imdb_path + 'plan_and_cost/train_plan_part{}.csv'.format(i)
    df = pd.read_csv(file)
    val_dfs.append(df)

val_df = pd.concat(val_dfs)

In [ ]:
# train.csv
#                         title t,movie_info_idx mi_idx                   t.id=mi_idx.movie_id             t.kind_id,=,7,mi_idx.info_type_id,>,99   283812
# 0                                             title t                                    NaN                           t.production_year,>,2004  1107925
# 1                                       movie_info mi                                    NaN                                mi.info_type_id,<,4  3624977
# 2                          title t,movie_companies mc                       t.id=mc.movie_id                                 mc.company_id,<,27   134807
# 3                                    movie_keyword mk                                    NaN                                 mk.keyword_id,<,55    54826
# 4            title t,movie_companies mc,movie_info mi      t.id=mc.movie_id,t.id=mi.movie_id  t.production_year,>,1977,mc.company_id,>,71403...  1172762
# ...                                               ...                                    ...                                                ...      ...
# 99994                        title t,movie_keyword mk                       t.id=mk.movie_id                               mk.keyword_id,>,1951  3102738
# 99995     title t,movie_companies mc,movie_keyword mk      t.id=mc.movie_id,t.id=mk.movie_id  t.kind_id,<,6,mc.company_type_id,<,2,mk.keywor...      738
# 99996     title t,movie_info mi,movie_info_idx mi_idx  t.id=mi.movie_id,t.id=mi_idx.movie_id  t.kind_id,=,7,t.production_year,=,2006,mi.info...   103764
# 99997  title t,movie_info_idx mi_idx,movie_keyword mk  t.id=mi_idx.movie_id,t.id=mk.movie_id  t.kind_id,=,7,t.production_year,<,1990,mk.keyw...        9
# 99998                                movie_keyword mk                                    NaN                               mk.keyword_id,<,3219  1881093

In [14]:
table_sample = get_job_table_sample(imdb_path+'train')

Loaded queries with len:  100000
Loaded bitmaps


In [15]:
table_sample[0]['title'].shape, table_sample[0]['movie_info_idx'].shape

((1000,), (1000,))

In [16]:
train_ds = PlanTreeDataset(full_train_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)
val_ds = PlanTreeDataset(val_df, None, encoding, hist_file, card_norm, cost_norm, to_predict, table_sample)

In [20]:
crit = nn.MSELoss()
model, best_path = train(model, train_ds, val_ds, crit, cost_norm, args)

Epoch: 0  Avg Loss: 4.196241921745241e-05, Time: 7.305432081222534
Median: 1.783678383506568
Mean: 5.131559662678631
Epoch: 20  Avg Loss: 1.01084798166994e-05, Time: 143.32891201972961
Median: 1.2352131044572183
Mean: 1.6981965700382236
Epoch: 40  Avg Loss: 7.465704073547386e-06, Time: 274.2282452583313
Median: 1.1758432809269403
Mean: 1.5358430316785883
Epoch: 60  Avg Loss: 6.542587687727064e-06, Time: 443.19748735427856
Median: 1.1402319216437744
Mean: 1.4586285542317845
Epoch: 80  Avg Loss: 5.591513130639214e-06, Time: 609.9643440246582
Median: 1.128797962964395
Mean: 1.42251704288672


In [ ]:
methods = {
    'get_sample' : get_job_table_sample,
    'encoding': encoding,
    'cost_norm': cost_norm,
    'hist_file': hist_file,
    'model': model,
    'device': args.device,
    'bs': 512,
}

In [ ]:
_ = eval_workload('job-light', methods)

In [ ]:
_ = eval_workload('synthetic', methods)